# **Multiclass Fish Image Classification**

# **Libraries Used**

In [1]:
import os
import numpy as np

from sklearn.metrics import classification_report

from tensorflow.keras.preprocessing import image
from tensorflow.keras import models,layers
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2, InceptionV3, EfficientNetB0
from tensorflow.keras.optimizers import Adam

# **Data Loading**

Train Data:

In [2]:
# Train Data path
train_data_path = "data/train"

# All Classes
classes = os.listdir(train_data_path)

# Ignore "Animal fish" and "Animal fish bass"
classes_filtered = [cls for cls in classes if cls.startswith("fish sea_food")]

# Keeping only Species name 
classes_new = [cls.replace("fish sea_food ", "") for cls in classes_filtered]
classes_final = [cls.replace("_", " ").title() for cls in classes_new]

# Rescaling
rescaled = image.ImageDataGenerator(rescale=1./255)

# Load Train Data
train_generator = rescaled.flow_from_directory(
    train_data_path,
    target_size=(224, 224),
    batch_size=32,
    classes=classes_filtered,
    class_mode="categorical"
)

Found 5099 images belonging to 9 classes.


Validation Data:

In [3]:
# Validation Data path
val_data_path = "data/val"

# Load Validation Data
val_generator = rescaled.flow_from_directory(
    val_data_path,
    target_size=(224, 224),
    batch_size=32,
    classes=classes_filtered,
    class_mode='categorical'
)

Found 895 images belonging to 9 classes.


Test Data:

In [4]:
# Test Data path
test_data_path = "data/test"

# Load Test Data
test_generator = rescaled.flow_from_directory(
    test_data_path,
    target_size=(224, 224),
    batch_size=32,
    classes=classes_filtered,
    class_mode='categorical',
    shuffle=False
)

Found 2654 images belonging to 9 classes.


Label Mapping:

In [5]:
# Label Mapping 
index_labeled = {i: name for i, name in enumerate(classes_final)}

# Class Mapped
print("Class index mapping:", index_labeled, sep='\n')

Class index mapping:
{0: 'Black Sea Sprat', 1: 'Gilt Head Bream', 2: 'Hourse Mackerel', 3: 'Red Mullet', 4: 'Red Sea Bream', 5: 'Sea Bass', 6: 'Shrimp', 7: 'Striped Red Mullet', 8: 'Trout'}


# **CNN Model**

Model Structure:

In [6]:
cnn_model = models.Sequential([
    layers.Input(shape=(224,224,3)),
    layers.Conv2D(32,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(9,activation='softmax'),
])

Model Compiler:

In [7]:
cnn_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Model Train:

In [8]:
cnn_trained = cnn_model.fit(
    train_generator, 
    epochs=5, 
    validation_data=val_generator
)

Epoch 1/5


c:\Users\vaisa\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


160/160 ━━━━━━━━━━━━━━━━━━━━ 63s 389ms/step - accuracy: 0.3798 - loss: 1.8514 - val_accuracy: 0.8648 - val_loss: 0.4255
Epoch 2/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 55s 343ms/step - accuracy: 0.8821 - loss: 0.3791 - val_accuracy: 0.8782 - val_loss: 0.3693
Epoch 3/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 54s 340ms/step - accuracy: 0.9578 - loss: 0.1311 - val_accuracy: 0.9385 - val_loss: 0.1749
Epoch 4/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 56s 349ms/step - accuracy: 0.9881 - loss: 0.0395 - val_accuracy: 0.9553 - val_loss: 0.1304
Epoch 5/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 58s 362ms/step - accuracy: 0.9949 - loss: 0.0185 - val_accuracy: 0.9743 - val_loss: 0.0765


Model Evaluate:

In [9]:
test_loss, test_accuracy = cnn_model.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

83/83 ━━━━━━━━━━━━━━━━━━━━ 8s 90ms/step - accuracy: 0.9678 - loss: 0.0863
Test Loss: 0.0882
Test Accuracy: 0.9691


Classification Report:

In [10]:
y_pred_probs = cnn_model.predict(test_generator)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = test_generator.classes
report = classification_report(y_true, y_pred, target_names=classes_final)
print(report)

83/83 ━━━━━━━━━━━━━━━━━━━━ 6s 75ms/step
                    precision    recall  f1-score   support

   Black Sea Sprat       0.98      0.98      0.98       298
   Gilt Head Bream       0.96      0.97      0.96       305
   Hourse Mackerel       0.95      0.95      0.95       286
        Red Mullet       0.99      0.98      0.98       291
     Red Sea Bream       0.97      0.96      0.97       273
          Sea Bass       0.96      0.93      0.95       327
            Shrimp       0.99      0.99      0.99       289
Striped Red Mullet       0.94      0.98      0.96       293
             Trout       0.99      0.99      0.99       292

          accuracy                           0.97      2654
         macro avg       0.97      0.97      0.97      2654
      weighted avg       0.97      0.97      0.97      2654



Image Test:

In [14]:
def predict_fish_class(img_path, model, label_map):
    img = image.load_img(img_path, target_size=(224, 224))    
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    predictions = model.predict(img_array)
    predicted_idx = np.argmax(predictions, axis=1)[0]
    predicted_name = label_map[predicted_idx]
    confidence = predictions[0][predicted_idx]
    
    return predicted_name, confidence

In [15]:
img_path = r'data\test\fish sea_food sea_bass\0L8PRYEJDGPN.jpg'
predicted_class, confidence_score = predict_fish_class(img_path, cnn_model, index_labeled)

print(f"Predicted Fish Species: {predicted_class}")
print(f"Confidence: {confidence_score:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted Fish Species: Sea Bass
Confidence: 0.93


In [16]:
img_path = r'data\test\fish sea_food red_mullet\0DTLG8H3NN5N.jpg'
predicted_class, confidence_score = predict_fish_class(img_path, cnn_model, index_labeled)

print(f"Predicted Fish Species: {predicted_class}")
print(f"Confidence: {confidence_score:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Predicted Fish Species: Red Mullet
Confidence: 1.00


Model Save:

In [17]:
cnn_model.save('CNN_Model.keras')

# **Pre-Trained Models**

Models:

In [18]:
pre_trained_models = [
    # {"name": "VGG16", "builder": VGG16, "preprocess": None},
    # {"name": "ResNet50", "builder": ResNet50, "preprocess": None},
    {"name": "InceptionV3", "builder": InceptionV3, "preprocess": None},
    {"name": "EfficientNetB0", "builder": EfficientNetB0, "preprocess": None},
    {"name": "MobileNetV2", "builder": MobileNetV2, "preprocess": None},
]

Model Trainings:

In [19]:
results = {}

for info in pre_trained_models:
    print(f"Training with {info['name']}")
    base_model = info["builder"](weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False

    # Build 
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dense(9, activation='softmax')
    ])

    # Compile
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train 
    history = model.fit(train_generator, epochs=5, validation_data=val_generator)

    # Evaluate
    test_loss, test_acc = model.evaluate(test_generator)
    print(f"{info['name']} Test Accuracy: {test_acc:.4f}")
    results[info['name']] = test_acc
    print()

Training with InceptionV3
Epoch 1/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 73s 433ms/step - accuracy: 0.7477 - loss: 0.7892 - val_accuracy: 0.9587 - val_loss: 0.1272
Epoch 2/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 70s 438ms/step - accuracy: 0.9843 - loss: 0.0723 - val_accuracy: 0.9788 - val_loss: 0.0688
Epoch 3/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 69s 432ms/step - accuracy: 0.9933 - loss: 0.0349 - val_accuracy: 0.9810 - val_loss: 0.0741
Epoch 4/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 65s 408ms/step - accuracy: 0.9971 - loss: 0.0222 - val_accuracy: 0.9765 - val_loss: 0.0822
Epoch 5/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 64s 398ms/step - accuracy: 0.9941 - loss: 0.0273 - val_accuracy: 0.9855 - val_loss: 0.0470
83/83 ━━━━━━━━━━━━━━━━━━━━ 29s 346ms/step - accuracy: 0.9917 - loss: 0.0285
InceptionV3 Test Accuracy: 0.9894

Training with EfficientNetB0
Epoch 1/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 70s 395ms/step - accuracy: 0.1055 - loss: 2.2208 - val_accuracy: 0.1095 - val_loss: 2.1972
Epoch 2/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 63s 394ms/step 

Model Results:

In [20]:
print("Summary of all models:")
for name, acc in results.items():
    print(f"{name}: {acc:.4f}")

Summary of all models:
InceptionV3: 0.9894
EfficientNetB0: 0.1100
MobileNetV2: 0.9966


# **MobileNet Model**

In [26]:
mn_model = model

In [27]:
test_generator.reset()
pred_probs = mn_model.predict(test_generator)
pred_labels = np.argmax(pred_probs, axis=1)
true_labels = test_generator.classes
report = classification_report(true_labels, pred_labels, target_names=classes_final)
print("Classification Report:\n")
print(report)

83/83 ━━━━━━━━━━━━━━━━━━━━ 20s 244ms/step
Classification Report:

                    precision    recall  f1-score   support

   Black Sea Sprat       1.00      0.99      0.99       298
   Gilt Head Bream       1.00      1.00      1.00       305
   Hourse Mackerel       1.00      1.00      1.00       286
        Red Mullet       1.00      0.99      0.99       291
     Red Sea Bream       1.00      1.00      1.00       273
          Sea Bass       0.99      1.00      1.00       327
            Shrimp       1.00      1.00      1.00       289
Striped Red Mullet       0.98      1.00      0.99       293
             Trout       1.00      1.00      1.00       292

          accuracy                           1.00      2654
         macro avg       1.00      1.00      1.00      2654
      weighted avg       1.00      1.00      1.00      2654



Image Test:

In [31]:
img_path = r'data\test\fish sea_food sea_bass\0L8PRYEJDGPN.jpg'
predicted_class, confidence_score = predict_fish_class(img_path, mn_model, index_labeled)

print(f"Predicted Fish Species: {predicted_class}")
print(f"Confidence: {confidence_score:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 692ms/step
Predicted Fish Species: Sea Bass
Confidence: 0.99


In [32]:
img_path = r'data\test\fish sea_food red_mullet\0DTLG8H3NN5N.jpg'
predicted_class, confidence_score = predict_fish_class(img_path, mn_model, index_labeled)

print(f"Predicted Fish Species: {predicted_class}")
print(f"Confidence: {confidence_score:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted Fish Species: Red Mullet
Confidence: 1.00


Model Save:

In [33]:
mn_model.save('MobileNet_Model.keras')